In [ ]:
# verified on 2/16/2025
# format of pass.txt
"""
google:your_email@gmail.com:your_google_app_password
yahoo:your_email@yahoo.com:your_yahoo_app_password
""";

In [ ]:
import imaplib
import email
from email.header import decode_header
import pandas as pd
import re

# Yahoo IMAP settings
IMAP_SERVER = 'imap.mail.yahoo.com'
IMAP_PORT = 993
# EMAIL = 'your_email@yahoo.com'
# PASSWORD = 'your_app_password'

with open('pass.txt') as fin:
    strs=fin.readlines()
for s in strs:
    if s.split(":")[0]=='yahoo':
        EMAIL=s.split(":")[1].strip()
        PASSWORD=s.split(":")[-1].strip()
# EMAIL,PASSWORD    

In [ ]:
# Connect to the server
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL, PASSWORD)

MAILBOX='INBOX'

status, _ = mail.select(MAILBOX)
if status != 'OK':
    print(f"Failed to select mailbox: {MAILBOX}")

In [ ]:
start_date = '01-Feb-2025'
end_date = '02-Feb-2025'

# Search for emails within the date range
status, messages = mail.uid('search',None, f'(SINCE {start_date} BEFORE {end_date})')
# status, messages = mail.search(None, f'SINCE "{start_date}"')
# status, messages = mail.search(None, 'ALL')

# sender_email = 'example@domain.com'
# # Search for emails from a particular address between dates
# search_criteria = f'(FROM "{sender_email}" SINCE "{start_date}" BEFORE "{end_date}")'
# status, messages = mail.search(None, search_criteria)

# Convert the message IDs to a list
target_uids = messages[0].split()

print(f"Found {len(target_uids)} emails.")


In [ ]:
results=[]
i=0
for uid in target_uids:
    i+=1
    # UID FETCH command
    status, data = mail.uid('fetch', uid, '(RFC822)')
    if status == 'OK' and data != [None]:
        r={}
        # Parse email content
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)
        sender=msg['From']
        print(f"\n{i} UID: {uid}")
        print(f"From: {sender}")
        
        # Decode headers
        subject, encoding = decode_header(msg['Subject'])[0]
        if isinstance(subject, bytes):
            try:
                subject = subject.decode(encoding if encoding else 'utf-8')
            except:
                subject="Decoding Error"        

        print(f"Subject: {subject}")
        
        r["id"]=uid
        r["sender"]=sender
        r["subject"]=subject
        results.append(r)           

    else:
        print(f"Email with UID {uid} not found")

In [ ]:
def get_email(str_sender):
    pattern = re.compile(r'<(.*?)>')
    match = re.search(pattern, str_sender)

    if match:
        # print("Found:", match.group())
        email=match.group(1)
    else:
        # print("Not found")
        email="NA"   

    sender, encoding = decode_header(str_sender)[0]
    if isinstance(sender, bytes):
        sender = sender.decode(encoding if encoding else 'utf-8')

    return email,sender   

In [ ]:
df=pd.DataFrame(results)
df[["email","sender"]]=df.apply(lambda r:get_email(r['sender']),axis=1,result_type='expand')

In [ ]:
df

In [ ]:
list(df.email.unique())

In [ ]:
dx=df.groupby("email").count().reset_index().sort_values(by="sender",ascending=True)
dx

In [ ]:
dx[dx.subject>3].email.values

In [ ]:
f2=['temu@updates.temuofficial.com', 
       'r.moore@first.AmericanTruthCast.com', 
       'insight@the.paybackinvestigators.com',
       'edmond.p@daily.smoothtradingarea.com',
       'move@like.dealscatchers.com', 
       'news@tradelikegorillas.com',
       'ruffus.white@net.richpeoplenetworks.com',
       'raymondparks@tmp.titanmediapro.com', 
       'ae@e.ae.com',
       'daily@news.greenforesttrading.com',
       'daily@the.bankerbull.com', 
       'daily@blog.secretprofittools.com',
       'day@s.longliveamerican.com', 
    #    'read@first.teamexpoexpert.com',
       'peter.s@the.directionforexpo.com', 
       'daily@inside.greenforesttrading.com',
       'l.barnett@wtf.wealthtimefuture.com',
       'email@digital.conservativeintel.com', 'gain@the.olympinvests.com',
       'hunter.wells@tip.profitabletradertips.com',
       'check@tips.sportinvestorsleague.com',
       'actual@Ii.investmentinnovate.com',
       'editor@nowcc.notesonwealthcreation.com',
       'editorial@wsi.wisetradeinvesting.com',
       'news@fn.fin-kit.com',
       'info@tfm.thefinancememories.com', 
       'news@daily.financiffy.com',
       'mailing@ceg.certaineconomicgrowth.com',
       'today@art.americansruletrading.com',
       'daily@mcp.moderncryptoparadise.com',
       'daily@d.tradetacticsnavigator.com',
    #    'j.blume@mc.metaversecapitalists.com',
       'news@from.thegreatassets.com', 
       'ronaldvin@a.capitaliststoday.com'],

In [ ]:
f1=['walmartcustomerexperience@express10.medallia.com',
    #    'fandango@movies.fandango.com',
       'ftowdklvjeqagcszbymr@plugins.homemakingguide.com',
       'gain@the.olympinvests.com', 
    #    'noreplyhr@fb.com',
    #    'Costco@digital.costco.com', 
       'raymondparks@tmp.titanmediapro.com',
       'read@first.teamexpoexpert.com',
       'e-news@email.bayareanewsgroup.com',
    #    'billpay@billpay.bankofamerica.com', 
    #    'no-reply@rentcafe.com',
    #    'onlinebanking@ealerts.bankofamerica.com',
    #    'store+28212789326@m.shopifyemail.com',
    #    'hello@news.expressvpn.com', 
    #    'NA', 
    #    'palmia@legacypartners.com',
       'editorial@wsi.wisetradeinvesting.com',
    #    'groupupdates@facebookmail.com', 
       'me@jasongee.site',
       'silhruettrgrzphsextup@plugins.homemakingguide.com',
       'contact@masterclassjournal.com',
       'notifications@d.lavendercollab.com', 
       'humble@pobriar.com',
       'kent.harper@news.theinvestingbox.com',
       'cutleryandmore@cutleryandmore.com', 
       'contact@lectricebikes.com',
       'trivia@mail.wordstrivia.com',
       'just@info.globalfinancechanges.com',
       'global@news.takeprofitalerts.com',
       'email@the.deliciousinvesting.com', 
       'reply-to@e.digikey.com',
    #    'hongxiachen@hotmail.com', 
       'p.lynch@hmg.honeymoneygazette.com',
       'daily@the.bankerbull.com', 
       'news@s.stripesandstarsnews.com',
       'mylife@mail.mylife.com', 
       'marketing@emails.hertz.com',
       'move@like.dealscatchers.com',
    #    'j.blume@mc.metaversecapitalists.com',
       'from@global.investingcongress.com',
       'editor@star.starinvestingship.com',
       't.killian@sotmm.sharksofthemarket.com']

In [ ]:

email_to_delete=f1+['blog.secretprofittools.com','pro.milkymoneyway.com','ldii.longdistanceinvestings.com','first.AmericanTruthCast.com','today.investingwithmind.com','day.usastocksinsider.com',"best.marketexpopro.com",'net.richpeoplenetworks.com','fin.finacetechfuture.com','editor@nowcc.notesonwealthcreation.com','prisintol.com','tfm.thefinancememories.com','today.investingwithmind.com','ucc.urbancapitalist.com','digest.marketbroadview.com','tips.sportinvestorsleague.com','cottonbubblelay.com','deals@deals.vegas.com','find@via.healthyinvestingtips.com','webinar@newsletters.techonline.com','l.barnett@wtf.wealthtimefuture.com','get@real.multitradersmoves.com','mailing@ceg.certaineconomicgrowth.com','temu@updates.temuofficial.com','strategy@insights.FinancesDataDriven.com','ruffus.white@net.richpeoplenetworks.com','with@top.goldmineadvisors.com','your@very.smartsuccessstrategy.com','news@daily.investmentjourneys.com','check@tips.sportinvestorsleague.com','daily@digest.marketbroadview.com','q.hayes@choice.besttoinvestnews.com','news@daily.financiffy.com','daily@blog.secretprofittools.com','mail@your.usagoldmiracle.com','the.directionforexpo.com','daily@digest.marketbroadview.com']


In [ ]:
subject_to_delete =['David Kant','FunWords Team','richard grant','stockpile saving','satelite earnings now','this token could','David Kant','Nextdish','hotels.com', 'Matthew Wehner','Elizabeth Harris', 'Trump','David Kant','Part-Time Work','Investors League','MileagePlus Partner','Russia in Shock','woot.com','Christopher Johnson','Tax-Free Cash Hack']

In [ ]:
pattern1 = '|'.join(subject_to_delete)
pattern2 = '|'.join(email_to_delete)

In [ ]:

dz=df[(df.sender.str.contains(pattern1, case=False)) | (df.email.str.contains(pattern2, case=False))]

In [ ]:
dz

In [ ]:
uids_to_delete=list(dz.id.unique())
uids_to_delete, len(uids_to_delete)

In [ ]:
# Delete emails
for uid in uids_to_delete:
    print(uid)
    try:
        status, response = mail.uid('store', uid, '+FLAGS', '\\Deleted')
        print(f"Delete status: {status}, Response: {response}")
    except:
        print(uid,"not working")
        
# Expunge deleted emails
mail.expunge()

In [ ]:

# Close connection
mail.close()
mail.logout()